In [55]:
import pathlib

path=str(pathlib.Path().resolve())+"/assets/"

In [59]:
import os
import numpy as np
import cv2 as cv
import random

def get_data(path):
    images = []  # Lista para las imágenes
    labels = []  # Lista para las etiquetas
    dirs = os.listdir(path)
    
    for i in os.listdir(path + "/" + dirs[0]):
        images.append(cv.imread(path + "/" + dirs[0] + "/" + i))
        labels.append(0)
    print(path + "/" + dirs[0])
    
    for i in os.listdir(path + "/" + dirs[1]):
        images.append(cv.imread(path + "/" + dirs[1] + "/" + i))
        labels.append(1)
    
    # Barajar las imágenes y las etiquetas de manera sincronizada
    combined = list(zip(images, labels))
    random.shuffle(combined)
    images, labels = zip(*combined)
    
    print(path + "/" + dirs[1])
    
    # Convertir las listas en arrays NumPy
    images_array = np.array(images)
    labels_array = np.array(labels)
    
    return images_array, labels_array


In [60]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor() as executor:
    f1 = executor.submit(get_data, path+"train")
    f2 = executor.submit(get_data, path+"test")
    f3 = executor.submit(get_data, path+"val")

    X_train,y_train = f1.result()
    X_test,y_test = f2.result()
    X_val,y_val = f3.result()


C:\Users\diego.guizanlopez\Documents\BigDataEIa\IA\Sistemas_Aprendizaje_Automatico\3-Trimestre\Redes_Neuronales\Clasificacion_Imagen\Ejercicio/assets/val/NORMAL
C:\Users\diego.guizanlopez\Documents\BigDataEIa\IA\Sistemas_Aprendizaje_Automatico\3-Trimestre\Redes_Neuronales\Clasificacion_Imagen\Ejercicio/assets/val/PNEUMONIA
C:\Users\diego.guizanlopez\Documents\BigDataEIa\IA\Sistemas_Aprendizaje_Automatico\3-Trimestre\Redes_Neuronales\Clasificacion_Imagen\Ejercicio/assets/test/NORMAL
C:\Users\diego.guizanlopez\Documents\BigDataEIa\IA\Sistemas_Aprendizaje_Automatico\3-Trimestre\Redes_Neuronales\Clasificacion_Imagen\Ejercicio/assets/test/PNEUMONIA
C:\Users\diego.guizanlopez\Documents\BigDataEIa\IA\Sistemas_Aprendizaje_Automatico\3-Trimestre\Redes_Neuronales\Clasificacion_Imagen\Ejercicio/assets/train/NORMAL
C:\Users\diego.guizanlopez\Documents\BigDataEIa\IA\Sistemas_Aprendizaje_Automatico\3-Trimestre\Redes_Neuronales\Clasificacion_Imagen\Ejercicio/assets/train/PNEUMONIA


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5216,) + inhomogeneous part.

In [ ]:
from matplotlib import pyplot as plt


class_names = [0,1]

n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(y_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

In [ ]:
y_train